# DataFrame Construction

## 1. Data Import

#### Importing `json` extension data
Received raw data was in json extension. You need to use `json` library to read it.

#### Loading the data
Each line in json file is in dictionary object so it has to be loaded each time as a column using `for` loop.
You then change it to `pd.DataFrame` object.

In [1]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

output = []

# Import STAT file in .json extension as a list
with open("2021-03-05_data_exchange/2021-02-12_2021-03-04_STAT_tripqueries.json", "r") as input:
    for line in input:
        output.append(json.loads(line))

# Convert input into DataFrame
df_moovizy = pd.DataFrame(output)

# Show all the column names
df_moovizy.columns

Index(['UserId', 'RequestId', 'RecordedAtTime', 'DepartureName',
       'DepartureCityName', 'DepartureCityCode', 'DeparturePointType',
       'DepartureStreetNumber', 'DepartureLatitude', 'DepartureLongitude',
       'ArrivalName', 'ArrivalCityName', 'ArrivalCityCode', 'ArrivalPointType',
       'ArrivalStreetNumber', 'ArrivalLatitude', 'ArrivalLongitude', 'ViaName',
       'ViaCityName', 'ViaCityCode', 'ViaPointType', 'ViaStreetNumber',
       'OptionsTargetTime', 'OptionsTargetTimeType', 'OptionsHeuristic',
       'OptionsMaxWalkDistance', 'OptionsWalkSpeed',
       'OptionsForbiddenOperators', 'OptionsForbiddenStops',
       'OptionsJourneyAttributes', 'OptionsAuthorizedTransportModes',
       'OptionsForbiddenLines', 'HasSolution', 'ResponseTimeAsMs', 'Solutions',
       'UnifiedTripModes', 'UnifiedSpecificOptions', 'UnifiedRequestId',
       'UnifiedPreferedTripMode', 'Trace', 'StatusCode', 'DetailedStatus',
       'ViaLatitude', 'ViaLongitude'],
      dtype='object')

## 2. Cleaning the data

#### Select columns that you wanna use.

In [2]:
# Select columns I need
df_moovizy_00 = df_moovizy[['RecordedAtTime', 'UserId', 'RequestId','UnifiedPreferedTripMode', 'Solutions',
                            'DepartureLatitude', 'DepartureLongitude', 'DepartureCityName', 'ArrivalCityName',
                            'OptionsHeuristic', 'OptionsMaxWalkDistance',
                            'OptionsWalkSpeed', 'OptionsJourneyAttributes', 'OptionsAuthorizedTransportModes',]]

#### Getting desired time data

`RecordedAtTime` column contains pieces of information such as `Date`, `Time`, `Reference`. You only need `Date` and `Time`. So you split `RecordedAtTime` into three different columns of `Date`, `Time`, `Reference` and then you drop `Reference` column.

And in the `Time` column, the time is expressed to miliseconds unit. You don't need that. Split this column into `Time` and `MiliSec` columns where `Time` column contains hour minute second info. Then you drop `Milisec` column.


In [3]:
df_moovizy_00[['Date','Time','TimeRef']] = pd.DataFrame(df_moovizy_00['RecordedAtTime'].str.split(' ', 2).tolist())
df_moovizy_00[['Time','MiliSec']] = pd.DataFrame(df_moovizy_00['Time'].str.split('.', 1).tolist())
df_moovizy_00 = df_moovizy_00.drop(['RecordedAtTime','MiliSec','TimeRef'],1)

# Just changing to shorter name
df_moovizy_00 = df_moovizy_00.rename(columns={'UnifiedPreferedTripMode':'PrefTripMode'})
df_moovizy_00.columns

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Index(['UserId', 'RequestId', 'PrefTripMode', 'Solutions', 'DepartureLatitude',
       'DepartureLongitude', 'DepartureCityName', 'ArrivalCityName',
       'OptionsHeuristic', 'OptionsMaxWalkDistance', 'OptionsWalkSpeed',
       'OptionsJourneyAttributes', 'OptionsAuthorizedTransportModes', 'Date',
       'Time'],
      dtype='object')

#### Column relocation

When you split a column, newly created columns go automatically to the right-end of the df. For better visualisation of data in the table, you may want to relocate the columns using their indices.

`df_moovizy_00.columns[-2:]` will take all the columns names except the last two.

`df_moovizy_00.columns[:-2]` will take the last two except the rest.

`df_moovizy_00_columns` will concatenate the two column names lists into one. Columns are now in the desired order.

`df_moovizy_00[df_moovizy_00_columns]` will relocate column names in the same order as the one above.

In [4]:
df_moovizy_00_col_right = df_moovizy_00.columns[-2:].to_list() 
df_moovizy_00_col_left  = df_moovizy_00.columns[:-2].to_list()
df_moovizy_00_columns   = df_moovizy_00_col_right + df_moovizy_00_col_left
df_moovizy_01 = df_moovizy_00[df_moovizy_00_columns]

#### Creating hour dummy

You need a time dummy variable that indicates each hour to separate hidden effects of unknown events happened in each hour on a given day.

In the `Time` column, there are request time data stored as "HH:MM:SS" format in string value. You only need the first two characters "HH" to know at which hour the request was made.

`df_moovizy_01['Time'].str[:2]` captures the first two str characters of all the observations of `Time` column.

Then `map(int, time_left)` changes all the str characters into integer.

`dt.timedelta(hours = h)` will change each `h` element in the list into seconds. `str()` function changes the seconds expressed in "HH:MM:SS" format automatically. Note that if the hour is not in double digit, for instance, 7 o'clock, will not be displayed as 07:00:00. It will be displayed like 7:00:00.

The following `for` loop is to store hours in `interval` list. It checks the length of the time string. If it's equal to 7, then the loop takes only the first letter of the string which is a set of non-double digit hours (0 to 9). If it's equal to 8, then the loop takes the first two letters of the string which are a set of double digit hours (10 to 23).

`interval2` changes `interval` to a df.

`pd.concat([df_moovizy_01, interval2], axis=1)` concatenates `interval2` to `df_moovizy_01`. Indexing doesn't matter here because from the beginning all the hours stored in the `interval` list is ordered in the same way as `df_moovizy_01`.

Do another column relocation.

In [5]:
time_left = np.array(df_moovizy_01['Time'].str[:2].tolist())
time_left = list(map(int, time_left))

left1 = list(time_left)

# Transform left parts time interval objects into str objects
left2 = []
for h in left1:
    left2.append(str(dt.timedelta(hours = h)))

# Joined time interval
interval = []
for i in range(len(right2)):
    if len(left2[i]) == 7:
        interval.append(left2[i][:1]) # Only the first letter like 8 (hour)
    if len(left2[i]) == 8:
        interval.append(left2[i][:2]) # Only the first two letters like 17 (hour)

# Create a DF with Interval column
interval2 = pd.DataFrame(interval, columns=["Interval"])

# Concatenate moovizy and interval DFs
df_moovizy_02 = pd.concat([df_moovizy_01, interval2], axis=1)

# Rearranging the order of columns
col_interval = df_moovizy_02.columns[-1:].to_list()  # interval column
col_datetime = df_moovizy_02.columns[:2].to_list()   # Until the 2nd column: Date and Time columns
col_the_rest = df_moovizy_02.columns[2:-1].to_list() # the rest
col_complete = col_datetime + col_interval + col_the_rest

# Give the order of columns to Moovizy DF
df_moovizy_02 = df_moovizy_02[col_complete]

The below cell is only to show how to create an interval so you can ignore

In [ ]:
# Time Interval Creation

# Left-end of the interval
time_left = np.array(df_moovizy_01['Time'].str[:2].tolist())
time_left = list(map(int, time_left))

# Right-end of the interval
ad = np.ones(len(time_left), dtype=np.int) # array of ones
time_right = time_left + ad                # +1 to each element of time_left

# Gonna change to DateTime object and it doesn't recog np.array so we change to lists
left1 = list(time_left)    # left  parts of the time interval stored as int
right1= list(time_right)   # right parts of the time interval stored as int

# Transform left parts time interval objects into str objects
left2 = []
for h in left1:
    left2.append(str(dt.timedelta(hours= h)))
    
right2 = []
for h in right1:
    # we need to change to int from np.int64 datatype cuz timedelta cannot read np.int then substract 1 sec
    right2.append(str(dt.timedelta(hours=np.int64.item(h)) - dt.timedelta(seconds= 1)))

# Joined time interval
interval = []
for i in range(len(right2)):
    if len(left2[i]) == 7:
        interval.append(left2[i][:1]) # Only the first letter like 8 (hour)
    if len(left2[i]) == 8:
        interval.append(left2[i][:2]) # Only the first two letters like 17 (hour)
        
# We can do ":00:00$"

# Create a DF with Interval column
interval2 = pd.DataFrame(interval, columns=["Interval"])

# Concatenate moovizy and interval DFs
df_moovizy_02 = pd.concat([df_moovizy_01, interval2], axis=1)
df_moovizy_02.head(2)

# Rearranging the order of columns
col_interval = df_moovizy_02.columns[-1:].to_list()  # interval column
col_datetime = df_moovizy_02.columns[:2].to_list()   # Until the 2nd column: Date and Time columns
col_the_rest = df_moovizy_02.columns[2:-1].to_list() # the rest
col_complete = col_datetime + col_interval + col_the_rest

# Give the order of columns to Moovizy DF
df_moovizy_02 = df_moovizy_02[col_complete]

#### Filtering Solutions Column

Get `Solutions` column separately by `df_moovizy_02[['Solutions']]`

`solutions_00[solutions_00['Solutions'].map(len) > 1]` takes only non-empty (length greater than 1) rows in the `Solutions` series. Some rows do not contain any solutions. You do not want those.

Separately from `Solutions` series, in the original dataset, you trim the dataset by indices of non-empty solutions rows with `solutions_01.index`. Trimming is done by `df_moovizy_02.iloc[idx_solutions,:]`.

Once the trimming is done, you have to reset the index from both solutions series and the original dataset in order to eliminate empty indices and to concatenate them. You can keep the original indices by setting `drop = False`.

In [6]:
###########################################################################################################
#                                                                                                         #
# Solutions column contains 2 to 3 possible routes in each there are Dist, Time, Dep. Time, Arr. Time etc #
#                                                                                                         #
###########################################################################################################

# Get Solutions column as a Series
solutions_00 = df_moovizy_02[['Solutions']]

# Getting rid of empty Solutions rows
solutions_01 = solutions_00[solutions_00['Solutions'].map(len) > 1] # this will leave some indices empty (ex: 1 2 5 6 8)

# Get indices of non-empty Solutions rows
idx_solutions = solutions_01.index # index object: not in 

# Choose rows of Moovizy DF that have the same indices as non-empty Solutions rows
df_moovizy_03 = df_moovizy_02.iloc[idx_solutions,:]
df_moovizy_04 = df_moovizy_03.reset_index(drop = True) # reset indices so that there are no empty skipped indices
solutions_02 = solutions_01.reset_index(drop = True)   # same but for solutions

In [7]:
# In Solutions column, a row contains a list of solutions.
# Each solution is a dictionary object.
# In each dictionary object there are multiple pieces of key info such as DepTime, ArrTime, Dist, Time etc

num_sol = []
for i in range(len(solutions_02)):
    num_sol.append(len(solutions_02.iloc[i,0]))

# Create empty DFs for each solution
df_sol_1 = pd.DataFrame(index=range(0,len(solutions_02)),
                        columns=['DepartureTime','ArrivalTime','DurationAsMinutes','LengthAsMeters',
                               'TransfersCount','TripSteps','Index'])

df_sol_2 = pd.DataFrame(index=range(0,len(solutions_02)),
                        columns=['DepartureTime','ArrivalTime','DurationAsMinutes','LengthAsMeters',
                               'TransfersCount','TripSteps','Index'])

df_sol_3 = pd.DataFrame(index=range(0,len(solutions_02)),
                        columns=['DepartureTime','ArrivalTime','DurationAsMinutes','LengthAsMeters',
                               'TransfersCount','TripSteps','Index'])


In [8]:
# Check how many routes were proposed
df_num_sol = pd.DataFrame(num_sol)
df_num_sol[0].value_counts()
# 3 routes:   205175
# 2 routes:     4131

3    205175
2      4131
Name: 0, dtype: int64

In [9]:
# List of travel info details stored as dict keys
keys = list(solutions_02.iloc[0,0][1].keys())

# Save each route in seaparate dataframes
for key in keys:
    for i in range(len(df_sol_1)):
        if num_sol[i] == 2:
            df_sol_1.loc[i,key] = solutions_02.iloc[i,0][0][key]
            df_sol_2.loc[i,key] = solutions_02.iloc[i,0][1][key]
        else:
            df_sol_1.loc[i,key] = solutions_02.iloc[i,0][0][key]
            df_sol_2.loc[i,key] = solutions_02.iloc[i,0][1][key]
            df_sol_3.loc[i,key] = solutions_02.iloc[i,0][2][key]

In [10]:
# Setting a new column order
df_sol_11 = df_sol_1.rename(columns={'DepartureTime': 'DepartureTime_1',
                                     'ArrivalTime': 'ArrivalTime_1',
                                     'DurationAsMinutes': 'DurationAsMinutes_1',
                                     'LengthAsMeters': 'LengthAsMeters_1',
                                     'TransfersCount': 'TransfersCount_1',
                                     'TripSteps': 'TripSteps_1',
                                     'Index': 'Route_1'})
df_sol_22 = df_sol_2.rename(columns={'DepartureTime': 'DepartureTime_2',
                                     'ArrivalTime': 'ArrivalTime_2',
                                     'DurationAsMinutes': 'DurationAsMinutes_2',
                                     'LengthAsMeters': 'LengthAsMeters_2',
                                     'TransfersCount': 'TransfersCount_2',
                                     'TripSteps': 'TripSteps_2',
                                     'Index': 'Route_2'})
df_sol_33 = df_sol_3.rename(columns={'DepartureTime': 'DepartureTime_3',
                                     'ArrivalTime': 'ArrivalTime_3',
                                     'DurationAsMinutes': 'DurationAsMinutes_3',
                                     'LengthAsMeters': 'LengthAsMeters_3',
                                     'TransfersCount': 'TransfersCount_3',
                                     'TripSteps': 'TripSteps_3',
                                     'Index': 'Route_3'})

# Combining all the solutions
df_sol = pd.concat([df_sol_11,df_sol_22,df_sol_33],axis=1)

In [11]:
# Split DepartureTime of route 1 into Date, Time, TimeReference
df_sol[['Departure_Date_1','Departure_Time_1','Departure_Ref_1']] = pd.DataFrame(df_sol['DepartureTime_1'].str.split(' ', 2).tolist())
# Drop DepartureTime column and TimeReference column
df_sol_dep1_01 = df_sol.drop(['DepartureTime_1','Departure_Ref_1'], axis=1)

# Get columns in the following order: Departure_Date_1, Departure_Time_1, and the rest
departure_time_cols_01 = df_sol_dep1_01.columns[-2:].to_list()
departure_rest_01  = df_sol_dep1_01.columns[:-2].to_list()
departure_both_01 = departure_time_cols_01 + departure_rest_01

# sort columns
df_sol_dep1_02 = df_sol_dep1_01[departure_both_01]
df_sol_dep1_02.head(2)

# Split ArrivalTime of route 1 into Date, Time, TimeReference
df_sol_dep1_02[['Arrival_Date_1','Arrival_Time_1','Arrival_Ref_1']] = pd.DataFrame(df_sol_dep1_02['ArrivalTime_1'].str.split(' ', 2).tolist())
# Drop ArrivalTime column and TimeReference column
df_sol_arr1_01 = df_sol_dep1_02.drop(['ArrivalTime_1','Arrival_Ref_1'], axis=1)

arrival_departure_01 = df_sol_arr1_01.columns[0:2].to_list()
arrival_time_cols_01 = df_sol_arr1_01.columns[-2:].to_list()
arrival_rest_01 = df_sol_arr1_01.columns[2:-2].to_list()
arrival_all_01 = arrival_departure_01 + arrival_time_cols_01 + arrival_rest_01

df_sol_arr1_02 = df_sol_arr1_01[arrival_all_01]

In [12]:
# Split DepartureTime of route 2 into Date, Time, TimeReference
df_sol_arr1_02[['Departure_Date_2','Departure_Time_2','Departure_Ref_2']] = pd.DataFrame(df_sol_arr1_02['DepartureTime_2'].str.split(' ', 2).tolist())
# Drop DepartureTime column and TimeReference column
df_sol_dep2_01 = df_sol_arr1_02.drop(['DepartureTime_2','Departure_Ref_2'], axis=1)

# Get columns in the following order: Departure_Date_2, Departure_Time_2, and the rest
route2_left  = df_sol_dep2_01.columns[:9].to_list()
route2_right = df_sol_dep2_01.columns[-2:].to_list()
route2_middle= df_sol_dep2_01.columns[9:-2].to_list()
route2_all   = route2_left + route2_right + route2_middle

# sort columns
df_dep_2 = df_sol_dep2_01[route2_all]

# Split ArrivalTime of route 1 into Date, Time, TimeReference
df_dep_2[['Arrival_Date_2','Arrival_Time_2','Arrival_Ref_2']] = pd.DataFrame(df_dep_2['ArrivalTime_2'].str.split(' ', 2).tolist())
# Drop ArrivalTime column and TimeReference column
df_arr_21 = df_dep_2.drop(['ArrivalTime_2','Arrival_Ref_2'], axis=1)

route2_arr_left = df_arr_21.columns[0:11].to_list()
route2_arr_mid  = df_arr_21.columns[11:-2].to_list()
route2_arr_right= df_arr_21.columns[-2:].to_list()
route2_arr_all  = route2_arr_left + route2_arr_right + route2_arr_mid

df_arr_22 = df_arr_21[route2_arr_all]

In [13]:
nan_index = []
for i, e in enumerate(list(df_arr_22['DepartureTime_3'])):
    if isinstance(e, float):
        nan_index.append(i)

len(nan_index) # 2797

deptime_3 = []
for i, e in enumerate(list(df_arr_22['DepartureTime_3'])):
    if isinstance(e, float):
        deptime_3.append('9999-01-01 00:00:00 UTC') # for empty solution rows
    else:
        deptime_3.append(e)
        
arrtime_3 = []
for i, e in enumerate(list(df_arr_22['ArrivalTime_3'])):
    if isinstance(e, float):
        arrtime_3.append('9999-01-01 00:00:00 UTC') # for empty solution rows
    else:
        arrtime_3.append(e)


In [14]:
df_arr_22.columns.get_loc('DepartureTime_3') # 18 >> column position is 18th (index)
df_arr_22.columns.get_loc('ArrivalTime_3')   # 19

#n = df_arr_22.columns[['DepartureTime_3']]
df_dep_30 = df_arr_22.drop(['DepartureTime_3','ArrivalTime_3'], axis=1)
df_dep_30['DepartureTime_3'] = deptime_3
df_dep_30['ArrivalTime_3'] = arrtime_3


# Split DepartureTime of route 3 into Date, Time, TimeReference
df_dep_30[['Departure_Date_3','Departure_Time_3','Departure_Ref_3']] = pd.DataFrame(df_dep_30['DepartureTime_3'].str.split(' ', 2).tolist())
# Drop DepartureTime column and TimeReference column
df_dep_31 = df_dep_30.drop(['DepartureTime_3','Departure_Ref_3'], axis=1)

df_dep_31[['Arrival_Date_3','Arrival_Time_3','Arrival_Ref_3']] = pd.DataFrame(df_dep_31['ArrivalTime_3'].str.split(' ', 2).tolist())
# Drop ArrivalTime column and TimeReference column
df_dep_32 = df_dep_31.drop(['ArrivalTime_3','Arrival_Ref_3'], axis=1)

In [15]:
route3_left = df_dep_32.columns[:18].to_list()
route3_mid  = df_dep_32.columns[18:-4].to_list()
route3_right= df_dep_32.columns[-4:].to_list()
route3_all = route3_left + route3_right + route3_mid

df_dep_33 = df_dep_32[route3_all]

In [16]:
# Drop Solutions column then concat df_dep_33. This will replace Solutions
df_moovizy_04_bis = df_moovizy_04.drop(['Solutions'], axis=1)
df_moovizy_05 = pd.concat([df_moovizy_04_bis,df_dep_33], axis=1)

In [17]:
# Get Time and Distance of the travel for each solution 1, 2 and 3
time_dist_1 = df_moovizy_05[['DurationAsMinutes_1', 'LengthAsMeters_1']]
time_dist_2 = df_moovizy_05[['DurationAsMinutes_2', 'LengthAsMeters_2']]
time_dist_3 = df_moovizy_05[['DurationAsMinutes_3', 'LengthAsMeters_3']]

# Make dist and time as int for Solution N.1
dist_1 = list(time_dist_1['LengthAsMeters_1'])
time_1 = list(time_dist_1['DurationAsMinutes_1'])
int_dist_1 = [ int(x) for x in dist_1 ]
int_time_1 = [ int(y) for y in time_1]

# Make dist and time as int for Solution N.2
dist_2 = list(time_dist_2['LengthAsMeters_2'])
time_2 = list(time_dist_2['DurationAsMinutes_2'])
int_dist_2 = [ int(x) for x in dist_2]
int_time_2  = [ int(y) for y in time_2]

# We only extract dist and time for Solution N.3 and don't change to int object bc some are NaN
dist_3 = list(time_dist_3['LengthAsMeters_3'])
time_3 = list(time_dist_3['DurationAsMinutes_3'])

In [18]:
# make it to str then change to int (bc its stored as str)
for i, x in enumerate(dist_3):
    if isinstance(x, str):
        dist_3[i] = int(x)

# bc of N/A objects we chose float
for i, x in enumerate(dist_3):
    if isinstance(x, float):
        dist_3[i] = 0
        
for i, x in enumerate(time_3):
    if isinstance(x, str):
        time_3[i] = int(x)

for i, x in enumerate(time_3):
    if isinstance(x, float):
        time_3[i] = 1

In [ ]:
# Speed computation for each solution and store each as a DF
speed_1 = pd.DataFrame(np.array(int_dist_1) / (np.array(int_time_1) * 60), columns=["Speed_1 (m/s)"])
speed_2 = pd.DataFrame(np.array(int_dist_2) / (np.array(int_time_2) * 60), columns=["Speed_2 (m/s)"])
speed_3 = pd.DataFrame(np.array(dist_3) / (np.array(time_3) * 60), columns=["Speed_3 (m/s)"])

# Concat the speed DFs as a single DF
speed = pd.concat([speed_1,speed_2,speed_3], axis=1)